#### Install reqs

In [1]:
# !pip install hyper-nn
# !pip install gym
# !pip install gym[box2d]
# !pip install tqdm
# !pip install tensorboard
# !pip install matplotlib
# !pip install celluloid
# !pip install ipympl
# !pip install JSAnimation
# !pip install optax
# !pip install tensorflow_probability
# !pip install einops

# Reinforce

In [1]:
# uncomment this to enable jax gpu preallocation, might lead to memory issues

import os
os.environ["XLA_PYTHON_CLIENT_PREALLOCATE"] = "false"

from typing import Sequence, List, Tuple, Optional

import numpy as np
import jax
import jax.numpy as jnp
import flax.linen as nn
import optax
import gym

from hypernn.jax.utils import count_jax_params


/home/shyam/anaconda3/envs/py39/lib/python3.9/site-packages/scipy/__init__.py:146: UserWarning: A NumPy version >=1.16.5 and <1.23.0 is required for this version of SciPy (detected version 1.23.3
  warnings.warn(f"A NumPy version >={np_minversion} and <{np_maxversion}"


In [9]:
from hypernn.jax import JaxLinearHyperNetwork

#### Target Policy Network

In [10]:
class MLP(nn.Module):

    @nn.compact
    def __call__(self, x):
        x = nn.Dense(256, use_bias=False)(x)
        x = nn.tanh(x)
        x = nn.Dense(256, use_bias=False)(x)
        x = nn.tanh(x)
        x = nn.Dense(4, use_bias=False)(x)
        return x

In [11]:
target_network = MLP()

count_jax_params(target_network, inputs=jnp.zeros((1,8)))

68608

#### Making a Lunar Lander Hypernetwork with a Custom Weight Generator

In [12]:
from typing import Any, Dict, List, Optional, Tuple, Type, Union  # noqa
import flax.linen as nn

class LunarLanderHypernetwork(JaxLinearHyperNetwork):

    def make_weight_generator(self):
        return nn.Sequential(
            [
                nn.Dense(32),
                nn.tanh,
                nn.Dense(self.weight_chunk_dim)
            ]
        )


### We can reduce the number of trainable parameters to less than 10%

In [13]:
EMBEDDING_DIM = 4
NUM_EMBEDDINGS = 512

hypernetwork = LunarLanderHypernetwork.from_target(
    target_network = target_network,
    embedding_dim = EMBEDDING_DIM,
    num_embeddings = NUM_EMBEDDINGS,
    inputs=jnp.zeros((1,8))
)
count_jax_params(hypernetwork, inputs=[jnp.zeros((1,8))])

/home/shyam/anaconda3/envs/py39/lib/python3.9/site-packages/flax/core/scope.py:740: FutureWarning: jax.tree_leaves is deprecated, and will be removed in a future release. Use jax.tree_util.tree_leaves instead.
  abs_value_flat = jax.tree_leaves(abs_value)
/home/shyam/anaconda3/envs/py39/lib/python3.9/site-packages/flax/core/scope.py:741: FutureWarning: jax.tree_leaves is deprecated, and will be removed in a future release. Use jax.tree_util.tree_leaves instead.
  value_flat = jax.tree_leaves(value)


6630

#### Tensorboard utils

In [14]:
import os
from torch.utils.tensorboard import SummaryWriter
from datetime import datetime

def get_tensorboard_logger(
    experiment_name: str, base_log_path: str = "tensorboard_logs"
):
    log_path = "{}/{}_{}".format(base_log_path, experiment_name, datetime.now())
    train_writer = SummaryWriter(log_path, flush_secs=10)
    full_log_path = os.path.join(os.getcwd(), log_path)
    print(
        "Follow tensorboard logs with: python -m tensorboard.main --logdir '{}'".format(full_log_path)
    )
    return train_writer


#### Rollout function

In [15]:
from tensorflow_probability.substrates import jax as tfp

import jax
import functools

@functools.partial(jax.jit, static_argnames=('apply_fn', 'method'))
def generate_params(apply_fn, hypernetwork_params, method):
    generated_params, _ = apply_fn({'params':hypernetwork_params}, method=method)
    return generated_params

@functools.partial(jax.jit, static_argnames=('apply_fn'))
def sample_actions(apply_fn, hypernetwork_params, obs, generated_params):
    out =  apply_fn({'params':hypernetwork_params}, jnp.expand_dims(jnp.array(obs), 0), generated_params=generated_params, has_aux=False)
    return out

def rollout(env, apply_fn, generate_params_fn, hypernetwork_params, render=False, seed: int = 0, env_seed=None) -> float:
    if env_seed is not None:
        # env.seed(env_seed)
        obs = env.reset(seed=env_seed)
    else:
        obs = env.reset()
    done = False
    observations, actions, rewards, rendereds = [], [], [], []
    generated_params = generate_params(apply_fn, hypernetwork_params, generate_params_fn)
    while not done:
        rendered = None
        if render:
            rendered = env.render(mode="rgb_array")
            rendereds.append(rendered)

        out = sample_actions(apply_fn, hypernetwork_params, obs, generated_params)
        out = jnp.squeeze(out)
        dist = tfp.distributions.Categorical(logits=out)
        rng = jax.random.PRNGKey(np.random.randint(0, 10**10))
        action = dist.sample(seed=rng).item()

        next_obs, r, done, _ = env.step(action)

        observations.append(obs)
        actions.append(action)
        rewards.append(r)

        obs = next_obs

    num_steps = len(observations)
    env.close()
    return observations, actions, rewards, rendereds, num_steps

#### Reinforce algorithm

In [16]:
import optax
from flax.training import train_state  # Useful dataclass to keep train state
import flax
import logging

# prevent a ton of warnings from https://github.com/tensorflow/probability/issues/1523
logger = logging.getLogger("root")


class CheckTypesFilter(logging.Filter):
    def filter(self, record):
        return "check_types" not in record.getMessage()


logger.addFilter(CheckTypesFilter())

def create_train_state(rng, hypernetwork, learning_rate, input_shape):
    """Creates initial `TrainState`."""
    params = hypernetwork.init(rng, [jnp.zeros(input_shape)])['params']

    tx = optax.chain(
        optax.clip_by_global_norm(100.0),
        # optax.scale_by_adam(),  # Use the updates from adam.
        # optax.scale_by_schedule(scheduler),  # Use the learning rate from the scheduler.
        # # Scale updates by -1 since optax.apply_updates is additive and we want to descend on the loss.
        # optax.scale(-1.0)
        optax.adam(learning_rate)
    )
    return train_state.TrainState.create(
        apply_fn=hypernetwork.apply, params=params, tx=tx)

@functools.partial(jax.jit, static_argnames=('apply_fn'))
def train_step(apply_fn, state, observations, actions, discounted_rewards):
    def loss_fn(params):
        logits = apply_fn({'params':params}, observations, has_aux=False)
        dist = tfp.distributions.Categorical(logits=jnp.squeeze(logits))
        log_probs = jnp.squeeze(dist.log_prob(actions))
        loss = jnp.sum(discounted_rewards * log_probs)
        return -1*loss
    grad_fn = jax.value_and_grad(loss_fn)
    loss, grads = grad_fn(state.params)
    return state.apply_gradients(grads=grads), loss, grads

import tqdm

import collections

def flatten(d, parent_key='', sep='_'):
    items = []
    for k, v in d.items():
        new_key = parent_key + sep + k if parent_key else k
        if isinstance(v, flax.core.frozen_dict.FrozenDict) or isinstance(v, dict):
            items.extend(flatten(v, new_key, sep=sep).items())
        else:
            items.append((new_key, v))
    return dict(items)

def discount_reward(rews, gamma: float = 0.99):
    n = len(rews)
    rtgs = np.zeros_like(rews)
    for i in reversed(range(n)):
        rtgs[i] = rews[i] + gamma*(rtgs[i + 1] if i + 1 < n else 0)
    return rtgs

def reinforce(
        num_epochs,
        env,
        hypernetwork,
        env_seed: int = None,
        seed: int = 0,
        lr: float = 0.0001,
        gamma: float = 0.99,
    ):
    writer = get_tensorboard_logger("HypernetworkJaxRL")

    rng = jax.random.PRNGKey(seed)
    rng, init_rng = jax.random.split(rng)
    state = create_train_state(init_rng, hypernetwork, lr, (1,8))

    bar = tqdm.tqdm(np.arange(num_epochs))

    try:
        for i in bar:
            observations, actions, rewards, _, num_steps = rollout(env, hypernetwork.apply, hypernetwork.generate_params, state.params, seed=seed, env_seed=env_seed)
            discounted_rewards = discount_reward(np.array(rewards), 0.99)
            discounted_rewards = discounted_rewards - np.mean(discounted_rewards)
            discounted_rewards = discounted_rewards / (
                np.std(discounted_rewards) + 1e-10
            )

            observations = jnp.array(observations)
            actions = jnp.array(actions)
            discounted_rewards = jnp.array(discounted_rewards)

            state, loss, grads = train_step(hypernetwork.apply, state, observations, actions, discounted_rewards)
            grad_dict = {k:dict(grads[k]) for k in grads.keys()}
            grad_dict = flatten(grad_dict)
            grad_dict = {k:np.sum(np.array(grad_dict[k])) for k in grad_dict}

            metrics = {"loss":loss.item(), "rewards":np.sum(rewards), "num_steps":num_steps, **grad_dict}

            for key in metrics:
                writer.add_scalar(key, metrics[key], i)

            bar.set_description('Loss: {}, Sum Reward: {}'.format(loss.item(), np.sum(rewards)))
        return hypernetwork, state
    except KeyboardInterrupt as e:
        return hypernetwork, state


In [17]:
from jax.lib import xla_bridge
print(xla_bridge.get_backend().platform)

cpu


In [18]:
import gym

env = gym.make("LunarLander-v2")

In [19]:
hyper, state = reinforce(5000, env, hypernetwork, seed=0, lr=0.0002, env_seed=None)

Follow tensorboard logs with: python -m tensorboard.main --logdir '/home/shyam/Code/hyper-nn/notebooks/reinforce/tensorboard_logs/HypernetworkJaxRL_2023-01-17 12:20:38.049707'


/home/shyam/anaconda3/envs/py39/lib/python3.9/site-packages/flax/core/scope.py:740: FutureWarning: jax.tree_leaves is deprecated, and will be removed in a future release. Use jax.tree_util.tree_leaves instead.
  abs_value_flat = jax.tree_leaves(abs_value)
/home/shyam/anaconda3/envs/py39/lib/python3.9/site-packages/flax/core/scope.py:741: FutureWarning: jax.tree_leaves is deprecated, and will be removed in a future release. Use jax.tree_util.tree_leaves instead.
  value_flat = jax.tree_leaves(value)
  0%|          | 0/5000 [00:00<?, ?it/s]/home/shyam/anaconda3/envs/py39/lib/python3.9/site-packages/flax/core/scope.py:740: FutureWarning: jax.tree_leaves is deprecated, and will be removed in a future release. Use jax.tree_util.tree_leaves instead.
  abs_value_flat = jax.tree_leaves(abs_value)
/home/shyam/anaconda3/envs/py39/lib/python3.9/site-packages/flax/core/scope.py:741: FutureWarning: jax.tree_leaves is deprecated, and will be removed in a future release. Use jax.tree_util.tree_leaves

In [1]:
import numpy as np
import matplotlib
import matplotlib.pyplot as plt
%matplotlib ipympl
# Imports specifically so we can render outputs in Jupyter.
from JSAnimation.IPython_display import display_animation
from matplotlib import animation
from IPython.display import display
from celluloid import Camera
from IPython.display import HTML


def render_rollout(model, state, seed=10):
    fig = plt.figure()
    camera = Camera(fig)
    ax = fig.add_subplot(111)
    observations, actions, rewards, rendereds, num_steps = rollout(gym.make("LunarLander-v2"), model.apply, model.generate_params, state.params, seed=seed, render=True)
    frames = []
    for r in rendereds:
        frame = ax.imshow(r)
        ax.axis('off')
        camera.snap()
        frames.append([frame])
    print("Sum Reward: {}".format(np.sum(rewards)))
    animation = camera.animate(interval=50)
    # display(animations.to_html5_video())
    animation.save('HyperJaxLunarLander.mp4')
    return animation

In [2]:
vid = render_rollout(hypernetwork, state, seed=0)

NameError: name 'hypernetwork' is not defined

##### Play video

In [64]:
from IPython.display import Video

Video("HyperJaxLunarLander.mp4")